In [4]:
from keras import layers
from keras.models import Sequential
from tensorflow.keras import optimizers
import keras_tuner as kt

In [22]:
def build_model(hp):
    model= Sequential()
    
    model.add(layers.Conv2D(32,(3,3),activation="relu", input_shape= (150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128,(3,3),activation="relu"))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(units=hp.Choice("layer size",[128,256,512]),activation="relu"))
              
    model.add(layers.Dense(1,activation="sigmoid"))


    model.compile(loss = "binary_crossentropy",
                  optimizer=optimizers.RMSprop(learning_rate= 1e-4),
                  metrics= ["acc"])
              
    return model

In [23]:
keras_tuner=kt.RandomSearch( build_model,
                             objective="val_acc",
                             overwrite=True,
                             max_trials=3,
                             directory="models/parameter_tuning",
                             project_name="parameter_tuning",
                            )

# data generation

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_directory="dataset/catdog/train"
validation_directory="dataset/catdog/validation"

train_generator= train_datagen.flow_from_directory(train_directory,
                                                   target_size=(150,150),
                                                   batch_size=20,
                                                   class_mode= "binary")

validation_generator= train_datagen.flow_from_directory(validation_directory,
                                                        target_size=(150,150),
                                                        batch_size=20,
                                                        class_mode= "binary")

Found 1999 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# keras tuner'ı çalıştırmaya başlayabiliriz.

In [ ]:
keras_tuner.search_space_summary()

keras_tuner.search(train_generator,
                   validation_data=validation_generator,
                   epochs=25)
best_model=keras_tuner.get_best_models()[0]

Trial 1 Complete [00h 22m 42s]
val_acc: 0.7609999775886536

Best val_acc So Far: 0.7609999775886536
Total elapsed time: 00h 25m 04s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
128               |512               |layer size

Epoch 1/25
100/100 [==============================] - 55s 531ms/step - loss: 0.6925 - acc: 0.5188 - val_loss: 0.6864 - val_acc: 0.6070
Epoch 2/25
100/100 [==============================] - 49s 485ms/step - loss: 0.6773 - acc: 0.5828 - val_loss: 0.6738 - val_acc: 0.5510
Epoch 3/25
100/100 [==============================] - 53s 527ms/step - loss: 0.6517 - acc: 0.6288 - val_loss: 0.6510 - val_acc: 0.6080
Epoch 4/25
100/100 [==============================] - 54s 537ms/step - loss: 0.6204 - acc: 0.6628 - val_loss: 0.6241 - val_acc: 0.6600
Epoch 5/25
100/100 [==============================] - 56s 550ms/step - loss: 0.5911 - acc: 0.6763 - val_loss: 0.6124 - val_acc: 0.6580
Epoch 6/25
100/100 [==============================] - 53s 535ms

In [ ]:
model.save("models/cat_dog_hp_tuning.h5")